In [29]:
"""
    STEP 1: IMPORTING PYTHON MODULES
"""

import numpy as np
import pandas as pd
from keras.layers import LSTM, Bidirectional, Dropout, Dense
from keras.models import Sequential
from os import listdir, mkdir

In [40]:
"""
    STEP 2: WRITING METHODS REGARDING CONVERTING MUSICAL PROPERTIES INTO DATA POINTS AND VICE VERSA
"""

notesToNum = {
    'C': 0,
    'D': 2,
    'E': 4,
    'F': 5,
    'G': 7,
    'A': 9,
    'B': 11
}

pitchMap = {
    'rest': -1,
    '[]': -1
}

for pitch in notesToNum:
    pitchMap[(pitch + '0')] = notesToNum[pitch]
    pitchMap[(pitch + 'b')] = (notesToNum[pitch] - 1) % 12
    pitchMap[(pitch + '#')] = (notesToNum[pitch] + 1) % 12
    pitchMap[(pitch + '-2')] = (notesToNum[pitch] - 2) % 12
    pitchMap[(pitch + '2')] = (notesToNum[pitch] + 2) % 12

decryptPitchSharp = {
    -1: '__',
    0: 'C0',
    1: 'C#',
    2: 'D0',
    3: 'D#',
    4: 'E0',
    5: 'F0',
    6: 'F#',
    7: 'G0',
    8: 'G#',
    9: 'A0',
    10: 'A#',
    11: 'B0'
}

decryptPitchFlat = {
    -1: '__',
    0: 'C0',
    1: 'Db',
    2: 'D0',
    3: 'Eb',
    4: 'E0',
    5: 'F0',
    6: 'Gb',
    7: 'G0',
    8: 'Ab',
    9: 'A0',
    10: 'Bb',
    11: 'B0'
}

chordMap = {
    '[]': 0,
    'nan': 0,
    'major': 1,
    'maj': 1,
    'major-seventh': 1,
    'maj7': 1,
    'major-sixth': 1,
    '6': 1,
    'major-ninth': 1,
    'maj9': 1,
    'maj69': 1,
    'minor': 2,
    'min': 2,
    'minor-seventh': 2,
    'min7': 2,
    'minor-sixth': 2,
    'minor-ninth': 2,
    'minor-11th': 2,
    'minor-13th': 2,
    'minor-major': 2,
    'minMaj7': 2,
    'dominant': 1,
    '7': 1,
    'dominant-ninth': 1,
    '9': 1,
    'dominant-11th': 1,
    'dominant-13th': 1,
    'diminished': 3,
    'dim': 3,
    'half-diminished': 3,
    'm7b5': 3,
    'diminished-seventh': 3,
    'dim7': 3,
    ' dim7': 3,
    'augmented': 4,
    'aug': 4,
    'augmented-seventh': 4,
    'augmented-ninth': 4,
    'suspended-fourth': 5,
    'sus47': 5,
    'suspended-second': 5,
    'power': 1,
    'major-minor': 1,

    0: '[]',
    1: ' ',
    2: 'm',
    3: 'dim',
    4: 'aug',
    5: 'sus',
}

keyToSemitonesFromC = {
        0: 0,
        1: 7,
        -1: 5,
        2: 2,
        -2: 10,
        3: 9,
        -3: 3,
        4: 4,
        -4: 8,
        5: 11,
        -5: 1,
        6: 6,
        -6: 6,
        7: 1,
        -7: 11,
    }


"""
    *** CHORD ROOT & NOTE PITCH columns will end up with INTEGERS

    note: a row of csv data corresponding to a note in a song
"""
def transposeToAllNatural(note):
    key = note[2]
    note[2] = 0

    note[6] = pitchMap[note[6]]

    if note[6] != -1:
        note[6] -= keyToSemitonesFromC[key]
        note[6] = (note[6]) % 12

    note[4] = pitchMap[note[4]]
    if note[4] != -1:
        note[4] -= keyToSemitonesFromC[key]
        note[4] = note[4] % 12

    try:
        note[5] = chordMap[note[5]]
    except KeyError:
        note[5] = -1


        
def transposeToOriginalKey(pitch, originalKey):
    sharpKey = originalKey > 0

    transpose = keyToSemitonesFromC[originalKey]
    pitch = (pitch + transpose) % 12
    
    if sharpKey:
        return decryptPitchSharp[pitch]
    
    return decryptPitchFlat[pitch]
    
    
"""
    Turn time signature into a numeric fraction
    e.g. 4/4 -> 1.0
         3/4 -> .75
         12/8 -> 1.5
"""
def timeSignAsFraction(timeSign):
    frac = 0
    slashIndex = timeSign.find('/')
    if slashIndex < 0:
        return 1
    
    frac = int(timeSign[:slashIndex]) / int(timeSign[slashIndex+1:])
    return frac

print('complete')

complete


In [42]:
"""
    STEP 3: PREPROCESSING TRAINING SET DATA
"""

def find_csv_filenames(path_to_dir, suffix=".csv"):
    filenames = listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith(suffix)]


# as always, change directory path as necessary
all_files = find_csv_filenames("dataset/csv_train")

melodyDatasets = np.zeros((len(all_files), 200, 12))
chordDatasets = np.zeros((len(all_files), 200, 60))
songLengths = []
        
for i in range(len(all_files)):
    df = pd.read_csv("dataset/csv_train/" + all_files[i])
    song = df.iloc[:, 0:9].values
    """
        Column 0 : time
        Column 1 : measure
    
        Column 2 : key_fifths
        Column 3 : key_mode
    
        Column 4 : chord_root
        Column 5 : chord_type
        
        Column 6 : note_root
        Column 7 : note_octave
        Column 8 : note_duration
    """
    songLengths.append(int(song[len(song)-1][1]))
    
    measureI = 0
    
    for note in song:
        transposeToAllNatural(note)
        timeSign = timeSignAsFraction(note[0])
        
        try:
            if note[4] == -1:
                continue
        except KeyError:
            continue

        if note[1] != measureI:
            if note[1] != 'X1':
                measureI = int(note[1])
                chord = 12*(note[5]-1) + note[4]
                chordDatasets[i, measureI-1, chord] = 1
            
        
        if note[6] >= 0:
            melodyDatasets[i, measureI-1, note[6]] += note[8] / timeSign
            
            
print('processing complete')

processing complete


In [43]:
"""
    STEP 4: BREAKING INPUT INTO SERIES OF 4-MEASURE SEQUENCES
"""

melodyTrain = []
chordTrain = []
measuresPerPhrase = 4

for songI in range(len(melodyDatasets)):
    for m in range(1000000000):
        if np.sum(chordDatasets[songI][m:m+measuresPerPhrase]) < 1:
            break
        melodyTrain.append(melodyDatasets[songI][m:m+measuresPerPhrase])
        chordTrain.append(chordDatasets[songI][m:m+measuresPerPhrase])
        
melodyTrain = np.array(melodyTrain)
chordTrain = np.array(chordTrain)

print(melodyTrain.shape)
print(chordTrain.shape)
print(melodyTrain[0])
print(chordTrain[0])

(72040, 4, 12)
(72040, 4, 60)
[[ 0.  0.  0.  0.  0.  0.  0.  4.  0.  0.  0.  0.]
 [ 0.  0.  2.  0. 14.  0.  0.  0.  0.  0.  0.  0.]
 [ 8.  0.  4.  0.  0.  0.  0.  4.  0.  0.  0.  0.]
 [ 6.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [44]:
"""
    STEP 5: CREATING BLSTM MODEL
"""

model = Sequential()

# input layer with 12 units (12 pitches of a note)
model.add(LSTM(units=12, return_sequences=True, input_shape=(4, 12)))

# 2 hidden layers with 128 units
model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=.2)))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=.2)))

# output layer with (12 chord roots) * (5 chord types) == 60 units total
model.add(Dense(units=60))

# COMPILE RNN
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(melodyTrain, chordTrain, epochs=30, batch_size=512)

model.summary()

Epoch 1/30
141/141 [==============================] - 16s 113ms/step - loss: 0.0130
Epoch 2/30
141/141 [==============================] - 18s 128ms/step - loss: 0.0119
Epoch 3/30
141/141 [==============================] - 18s 130ms/step - loss: 0.0117
Epoch 4/30
141/141 [==============================] - 17s 121ms/step - loss: 0.0115
Epoch 5/30
141/141 [==============================] - 17s 123ms/step - loss: 0.0114
Epoch 6/30
141/141 [==============================] - 18s 127ms/step - loss: 0.0114
Epoch 7/30
141/141 [==============================] - 17s 122ms/step - loss: 0.0113
Epoch 8/30
141/141 [==============================] - 17s 119ms/step - loss: 0.0112
Epoch 9/30
141/141 [==============================] - 17s 121ms/step - loss: 0.0112
Epoch 10/30
141/141 [==============================] - 17s 121ms/step - loss: 0.0112
Epoch 11/30
141/141 [==============================] - 17s 121ms/step - loss: 0.0111
Epoch 12/30
141/141 [==============================] - 17s 123ms/step - lo

In [45]:
"""
    STEP 6: PREPROCESSING TEST DATA SET
"""

# as always, change directory path as necessary
all_files = find_csv_filenames("dataset/csv_test")
        

melodyTestDatasets = np.zeros((len(all_files), 200, 12))
chordTestDatasets = np.zeros((len(all_files), 200, 60))
songLengths = []
leadinLengths = []
originalSongKeys = []
        
for i in range(len(all_files)):
    df = pd.read_csv("dataset/csv_test/" + all_files[i])
    song = df.iloc[:, 0:9].values
    """
        Column 0 : time
        Column 1 : measure
    
        Column 2 : key_fifths
        Column 3 : key_mode
    
        Column 4 : chord_root
        Column 5 : chord_type
        
        Column 6 : note_root
        Column 7 : note_octave
        Column 8 : note_duration
    """
    
    songLengths.append(int(song[len(song)-1][1]))
    originalSongKeys.append(int(song[1][2]))
    
    measureI = 0
    leadin = 0
    
    for note in song:
        transposeToAllNatural(note)
        timeSign = timeSignAsFraction(note[0])
        
        
        if note[1] != measureI:
            if note[1] != 'X1':
                measureI = int(note[1])
                
                try:
                    if note[4] == -1:
                        leadin += 1
                    else:
                        chord = 12*(note[5]-1) + note[4]
                        chordTestDatasets[i, measureI-1, chord] = 1
                except KeyError:
                    leadin += 1
            
        
        if note[6] >= 0:
            melodyTestDatasets[i, measureI-1, note[6]] += note[8] / timeSign
            
    leadinLengths.append(leadin)

print(leadinLengths[:4])


melodyTest = []
chordTest = []
measuresPerPhrase = 4


for songI in range(len(melodyTestDatasets)):
    for m in range(1000000000):
        if m >= songLengths[songI]:
            break
        melodyTest.append(melodyTestDatasets[songI][m:m+measuresPerPhrase])
        chordTest.append(chordTestDatasets[songI][m:m+measuresPerPhrase])
        
melodyTest = np.array(melodyTest)
chordTest = np.array(chordTest)

print(len(all_files))
print(melodyTest.shape)
print(chordTest.shape)

[0, 6, 0, 1]
450
(18845, 4, 12)
(18845, 4, 60)


In [46]:
"""
    STEP 7: GET THE BLSTM MODEL TO PREDICT CHORDS
"""

suggestedChords = model.predict(melodyTest)

print(suggestedChords.shape)
    

(18845, 4, 60)


In [47]:
"""
    STEP 8: WRITE METHODS DETERMINING THE INDEX OF THE MAX ELEMENT
"""

def trueIndex(oneHotArray):
    if oneHotArray[0] == 1:
        return 0
    
    for i in range(1, len(oneHotArray)):
        if oneHotArray[i] == 1:
            return i
        
    return -1


def maxIndex(arr):
    maxI = 0
    maxV = arr[0]
    for i in range(1, len(arr)):
        if arr[i] > maxV:
            maxI = i
            maxV = arr[i]
    return maxI


print(trueIndex(chordTest[0, 0]))
print(suggestedChords[0, 0])
print(maxIndex(suggestedChords[0, 0]))

21
[ 1.0043646e-01  2.1694447e-03  9.9393710e-02 -1.3361271e-03
 -2.2157390e-02  2.3927785e-01  5.2586356e-03  1.5265845e-02
  1.1728035e-02  3.1796563e-03  2.4010055e-04  2.5850441e-04
 -3.4149792e-03  4.5681577e-03  7.2961718e-02 -2.4581419e-03
 -1.4225172e-02 -3.5273973e-03  1.6910934e-03  7.8664832e-03
 -6.8009319e-04  4.5597181e-01  1.9017353e-03 -3.9770240e-03
  2.3659358e-03 -1.5381163e-03 -2.1282132e-03  1.3955532e-02
 -2.3392909e-03 -1.2755389e-03  3.7789391e-04  1.7056649e-03
  1.8516861e-04  3.3779088e-03 -2.9287918e-04  3.3319350e-03
  1.0645133e-03  3.8518541e-04 -1.2697885e-04 -1.0770438e-03
 -2.3583928e-04  1.1255776e-03 -3.3183384e-04 -6.0160086e-04
 -1.6175988e-03  4.5678420e-03 -1.2820859e-03 -4.7595240e-04
 -4.5638308e-03  2.0077094e-05  1.0933480e-03  1.4366893e-04
 -3.8890459e-04 -4.5155566e-03  7.5157173e-04  3.4841087e-03
 -5.6196688e-05  1.1197793e-03  5.3018850e-04 -1.8208115e-03]
21


In [48]:
"""
    STEP 9: READ OUTPUT OF MODEL & MEASURE ACCURACY
"""

numSequences = suggestedChords.shape[0]
output = []
actual = []

correct = 0
total = 0

for i in range(numSequences):
    measuresPerPhrase = suggestedChords.shape[1]
    for m in range(measuresPerPhrase):
        predicted = maxIndex(suggestedChords[i, m])
        output.append(predicted)
        
        realChord = trueIndex(chordTest[i, m])
        actual.append(realChord)
        
        if realChord == predicted:
            correct += 1
        total += 1
        
        
            
            
print('correctly predicted:', correct, '/', total)
print('accuracy:', correct/total)
print(output[0:16])
print(actual[0:16])

correctly predicted: 35382 / 75380
accuracy: 0.4693817988856461
[21, 21, 14, 7, 21, 14, 7, 21, 7, 7, 21, 21, 21, 21, 21, 14]
[21, 0, 7, 4, 0, 7, 4, 21, 7, 4, 21, 0, 4, 21, 0, 7]


In [65]:
"""
    STEP 10: CONVERT OUTPUT BACK TO LIST OF CHORDS
"""

measuresPerPhrase = suggestedChords.shape[1]
predictions = []
predictionsEncrypted = []
real = []
realEncrypted = []
bigI = 0

correct = 0
total = 0

for songI in range(len(all_files)):
    predictedChords = []
    predictedEncryptedChords = []
    actualChords = []
    actualEncryptedChords = []
    
    for m in range(songLengths[songI]):
        chord = output[bigI]
        chordPitch = chord % 12
        chordType = chordMap[chord // 12 + 1]
        predictedEncryptedChords.append(chord)
        
        chord = (transposeToOriginalKey(chordPitch, originalSongKeys[songI]), chordType)
        predictedChords.append(chord)
        
        
        realChord = actual[bigI]
        realPitch = realChord % 12
        realType = chordMap[realChord // 12 + 1]
        actualEncryptedChords.append(realChord)
        
        if realType == '[]':
            realChord = ('[]', '[]')
        else:
            realChord = (transposeToOriginalKey(realPitch, originalSongKeys[songI]), realType)
            
        actualChords.append(realChord)
        
        
        bigI += measuresPerPhrase
        
        if chord == realChord:
            correct += 1
        total += 1

        
    predictions.append(predictedChords)
    predictionsEncrypted.append(predictedEncryptedChords)
    real.append(actualChords)
    realEncrypted.append(actualEncryptedChords)
    
    
print('correctly predicted:', correct, '/', total)
print('accuracy:', correct/total)

print(realEncrypted[0])

correctly predicted: 9072 / 18845
accuracy: 0.4814009020960467
[21, 0, 7, 4, 21, 0, 7, 4, 21, 0, 7, 4, 21, 0, 7, 4, 21, 0, 7, 4, 21, 0, 7, 4, 21, 0, 7, 4, 5, 7, 0, 21, 5, 7, 4, 4, 0, 4, 21, 14, 5, 7, 4, 4, 0, 4, 21, 14, 5, 7, 4, 4, 21, 0, 7, 4, 21, 0, 7, 4, 5, 7, 0, 21, 5, 7, 4, 4, 0, 4, 21, 14, 5, 7, 4, 4, 0, 4, 21, 14, 5, 7, 4, 4, 21, 0, 7, 4, 21, 0, 7, 4, 4, 0, 4, 21, 14, 5, 7, 4, 4, 0, 4, 21, 14, 5, 7, 4, 7, 0, 4, 21, 2, 5, 7, 4, 7, 0, 4, 21, 5, 7, 4]


In [66]:
"""
    STEP 11: OUTPUT THE LIST OF PREDICTED CHORDS TO .csv FILES
"""

def find_csv_filenames(path_to_dir, suffix=".csv"):
    filenames = listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith(suffix)]


# as always, change directory path as necessary
all_files = find_csv_filenames("dataset/csv_test")

for songI in range(len(all_files)):
    df = []
    
    for m in range(songLengths[songI]):
        chord = predictions[songI][m][0] + predictions[songI][m][1]
        if chord[1] == '0':
            chord = chord[0] + chord[2:]
            
        realChord = real[songI][m][0] + real[songI][m][1]
        if realChord[1] == '0':
            realChord = realChord[0] + realChord[2:]
            
        df.append([str(m+1), chord, realChord])
        
    outputFile = pd.DataFrame(df, columns=['Measure', 'Chord', 'Actual (for comparison only)'])
    outputFile.to_csv('dataset/csv_test/predictions/pred_' + all_files[songI])
    
newlyWrittenFiles = find_csv_filenames("dataset/csv_test/predictions")
print(len(newlyWrittenFiles))

450


In [72]:
"""
    STEP 12: GENERATE .midi FILES OF (MELODY + PREDICTED CHORDS) WITH LISA'S SUPPLEMENTARY CODE
"""

from midiutil import MIDIFile


chord_template_minor = np.array([0,3,7,12])
chord_template_major = np.array([0,4,7,12])
chord_template_suspended = np.array([0,5,7,12])
chord_template_diminished = np.array([0,3,6,12])
chord_template_augmented = np.array([0,4,8,12])

def write_midi(df, chords, songName):
    MyMIDI = MIDIFile(2)
    MyMIDI.addTempo(0, 0, 100)
    MyMIDI.addTempo(1, 0, 100)
    current_location = 0
    num_time_sig = int(df["time"][0].split("/")[0])
    denom_time_sig = int(df["time"][1].split("/")[0])
    MyMIDI.addTimeSignature(0,0,num_time_sig,2,24)
    MyMIDI.addTimeSignature(1,0,num_time_sig,2,24)
#     print(df["time"][0])
    for ri, row in enumerate(df["measure"]):
        row_index = ri
        time_sig_mod = int(df["time"][row_index].split("/")[1]) / int(df["time"][row_index].split("/")[0]) 
        time_sig_mod = 1/(denom_time_sig)
#         print("mod ", time_sig_mod)
        beat_duration = float(df["note_duration"][row_index])*time_sig_mod
        if df["note_root"][row_index] == "rest":
            current_location += beat_duration
            continue
        midi_num = (pitchMap[df["note_root"][row_index]] + int(df["key_fifths"][row_index])*7)%12 + 12*int(df["note_octave"][row_index])
#         print("beat_duration ", beat_duration)
        MyMIDI.addNote(0,0,midi_num,current_location,beat_duration,100)
        current_location += beat_duration
    
    current_location = 0
    for chord in chords:
#         print("CHORD: ", chord)
        if chord // 12 == 0:
            notes = chord_template_major + chord + 36
        elif chord // 12 == 1:
            notes = chord_template_minor + chord + 36
        elif chord // 12 == 2:
            notes = chord_template_diminished + chord + 36
        elif chord // 12 == 3:
            notes = chord_template_augmented + chord + 36
        else:
            notes = chord_template_suspended + chord + 36
            
        for note in notes:
#             print("note: ", note)
            MyMIDI.addNote(1,0,int(note),current_location,num_time_sig,100)
        current_location += num_time_sig
        
        
    with open("test_midi/" + songName + ".midi", "wb") as output_file:
        MyMIDI.writeFile(output_file)
                              
bigI = 0
for i in range(2):
    midiOutputTest = pd.read_csv('dataset/csv_test/' + all_files[i])
    write_midi(midiOutputTest, predictionsEncrypted[i], 'pred_' + all_files[i])
    write_midi(midiOutputTest, realEncrypted[i], 'real_' + all_files[i])
    bigI = songLengths[i]
    print(i+1, "songs processed")

1 songs processed


KeyError: '660'